In [19]:
import pandas as pd  # Import pandas library for data manipulation

# Define the file path (Update it based on your system)
file_path = r"C:\Users\nikit\Downloads\DATASET.csv"  

# Load the dataset into a pandas DataFrame
df = pd.read_csv(file_path)

# Display basic information about the dataset
print("Dataset Information:")
print(df.info())

# Display first few rows of the dataset
print("\nFirst 5 rows of the dataset:")
print(df.head())


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326921 entries, 0 to 326920
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   event_id            326921 non-null  int64  
 1   session_id          326921 non-null  int64  
 2   user_id             326921 non-null  int64  
 3   variation           326921 non-null  int64  
 4   platform            326921 non-null  object 
 5   datetime_event      326921 non-null  object 
 6   event_type          326921 non-null  object 
 7   final_order_status  170632 non-null  object 
 8   shop_id             315842 non-null  float64
dtypes: float64(1), int64(4), object(4)
memory usage: 22.4+ MB
None

First 5 rows of the dataset:
   event_id  session_id  user_id  variation platform  \
0  63527610     6391574   762832          2  Android   
1  12446736     6391574   762832          2  Android   
2  90232698     6391574   762832          2  Android 

In [20]:
# Convert 'datetime_event' column to datetime format for time-based analysis
df["datetime_event"] = pd.to_datetime(df["datetime_event"], errors="coerce")
print("\nBefore handling missing values, dataset shape:", df.shape)
# Check for missing values
print("\nMissing Values in Each Column:")
print(df.isnull().sum())

# Drop missing values if necessary
df = df.dropna(subset=["final_order_status"])  # Drop rows where final_order_status is missing

print("\nAfter handling missing values, dataset shape:", df.shape)



Before handling missing values, dataset shape: (326921, 9)

Missing Values in Each Column:
event_id                   0
session_id                 0
user_id                    0
variation                  0
platform                   0
datetime_event             0
event_type                 0
final_order_status    156289
shop_id                11079
dtype: int64

After handling missing values, dataset shape: (170632, 9)


In [21]:
# Count the number of unique users in each A/B test variation
user_distribution = df.groupby("variation")["user_id"].nunique()

# Print user distribution with proper labels
print("\nUser Distribution in Each Variation:")
print(f"Control Group (Old Design) - Variation 1: {user_distribution[1]} users")
print(f"Test Group (Larger Images) - Variation 2: {user_distribution[2]} users")




User Distribution in Each Variation:
Control Group (Old Design) - Variation 1: 27568 users
Test Group (Larger Images) - Variation 2: 14770 users


In [22]:
# Count unique users who placed at least one order in each variation group
orders_per_variation = df[df["event_type"] == "order_paid"].groupby("variation")["user_id"].nunique()

# Calculate conversion rate correctly
conversion_rates = (orders_per_variation / user_distribution) * 100

# Print conversion rates with labels
print("\nConversion Rates in Each Variation:")
print(f"Control Group (Old Design) - Variation 1: {conversion_rates[1]:.2f}%")
print(f"Test Group (Larger Images) - Variation 2: {conversion_rates[2]:.2f}%")



Conversion Rates in Each Variation:
Control Group (Old Design) - Variation 1: 100.00%
Test Group (Larger Images) - Variation 2: 100.00%


In [23]:
# Count unique users in the dataset
total_users = df["user_id"].nunique()

# Count users who placed at least one order
users_with_orders = df[df["event_type"] == "order_paid"]["user_id"].nunique()

# Print the results
print(f"\nTotal unique users in dataset: {total_users}")
print(f"Total users who placed at least one order: {users_with_orders}")
# Step 4: Ensure potential_buyers is defined before Step 5
# Count users who entered a restaurant (potential buyers)
potential_buyers = df[df["event_type"] == "entry_to_shop"].groupby("variation")["user_id"].nunique()

# Count users who placed an order
orders_per_variation = df[df["event_type"] == "order_paid"].groupby("variation")["user_id"].nunique()




Total unique users in dataset: 42338
Total users who placed at least one order: 42338


In [24]:
from scipy.stats import chi2_contingency

# Create a contingency table for chi-square test
contingency_table = pd.DataFrame({
    "Ordered": orders_per_variation,
    "Did Not Order": potential_buyers - orders_per_variation  # Users who visited but did not order
})

# Print contingency table before running the test
print("\nContingency Table:")
print(contingency_table)

# Fix: Add a small value to prevent zero division errors
chi2, p, dof, expected = chi2_contingency(contingency_table + 1e-10)

# Print results
print(f"\nChi-Square Test Result:")
print(f"Chi-Square Value: {chi2}")
print(f"p-value: {p}")

# Check if the result is statistically significant
if p < 0.05:
    print("The difference in conversion rates is statistically significant.")
else:
    print("The difference in conversion rates is NOT statistically significant.")





Contingency Table:
           Ordered  Did Not Order
variation                        
1            27568              0
2            14770              0

Chi-Square Test Result:
Chi-Square Value: 0.0
p-value: 1.0
The difference in conversion rates is NOT statistically significant.


In [25]:
# Count order outcomes per variation group
order_status_counts = df[df["event_type"] == "order_finished"].groupby(["variation", "final_order_status"])["user_id"].nunique().unstack()

# Convert to percentages for better comparison
order_status_percentages = (order_status_counts.div(order_status_counts.sum(axis=1), axis=0)) * 100

# Print order outcome percentages
print("\nOrder Outcome Breakdown (in %):")
print(order_status_percentages)



Order Outcome Breakdown (in %):
final_order_status  cancelled  refunded_after_delivery  successful
variation                                                         
1                   11.361775                 7.034130   81.604096
2                   12.066051                 1.098687   86.835262


In [26]:
# Count successful orders per platform and variation
platform_success_rates = df[(df["event_type"] == "order_finished") & (df["final_order_status"] == "successful")]\
    .groupby(["variation", "platform"])["user_id"].nunique().unstack()

# Count total users per platform
platform_user_counts = df.groupby(["variation", "platform"])["user_id"].nunique().unstack()

# Calculate success rate per platform
platform_conversion_rates = (platform_success_rates / platform_user_counts) * 100

# Print platform-specific success rates
print("\nSuccess Rate by Platform:")
print(platform_conversion_rates)



Success Rate by Platform:
platform     Android        iOS
variation                      
1          86.572250  87.070339
2          90.132563  91.070298
